# Table of Contents
1. [Definint MonteCarloOptionPricing Class](#Introduction)
2. [Producing *American put prices* similar to *Simulated American* in table 1 in Longstaff.
 (to ensure correct implmenetation)-](#Data-Loading-and-Preparation)
3. [Finite Difference results (benchmark for analysis)](#Exploratory-Data-Analysis)


# 1. Class

In [53]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats

class MonteCarloOptionPricing:
    def __init__(self, r, S0: float, K: float, T: float, σ: float, dim: int, n: int, seed: int, use_AV: bool = False):
        """ Class for pricing American Options using LSM. 
        
        Parameters: 
        r (float): Risk-free interest rate
        S0 (float): Initial asset value
        K (float): Strike price
        T (float): Time to maturity, in years
        σ (float): Volatility coefficient for diffusion
        dim (int): Number of paths to simulate
        n (int): Number of time steps between time 0 and time T
        seed (int): Random seed for reproducibility
        use_AV (bool): Flag to use Antithetic Variates method (default: False)
        """
        
        assert σ >= 0, 'Volatility cannot be less than zero'
        assert S0 >= 0, 'Initial stock price cannot be less than zero'
        assert T >= 0, 'Time to maturity cannot be less than zero'
        assert n >= 0, 'Number of slices per year cannot be less than zero'
        assert dim >= 0, 'Number of simulation paths cannot be less than zero'
        
        # Set the random seed for reproducibility
        np.random.seed(seed)
        
        self.r = r
        self.S0 = S0
        self.K = K
        self.T = T
        self.σ = σ
        self.n = n
        self.Δ = self.T / self.n
        self.df = np.exp(-self.r * self.Δ)
        self.use_AV = use_AV
        self.dim = dim

        if use_AV:
            assert dim % 2 == 0, 'For AV, the number of paths (dim) must be even'
            half_dim = dim // 2
            Z_half = np.random.normal(0, 1, (half_dim, self.n - 1)) #Z_half matrix with dimension (half_dim, self.n-1), representing random increments of the asset´s price over time for half the paths'
            self.Z = np.concatenate((Z_half, -Z_half), axis=0)  # Antithetic variates. Creating full matrix self:Z by concatenating Z_half with its negation -Z.half.
        else:
            self.Z = np.random.normal(0, 1, (dim, self.n - 1))  # Original method

        self.S = np.full((dim, self.n), np.nan)  # Allocate space for stock price process

    def GeometricBrownianMotion(self):
        """ Generate GBM paths according to Algorithm 3.
        
        Returns:
        np.ndarray: Simulated paths of the asset price.
        """
        
        # unpack parameters
        Δ = self.Δ
        Z = self.Z
        S = self.S
        S0 = self.S0
        r = self.r
        σ = self.σ
        n = self.n
        
        S[:,0] = np.log(S0)  # Set initial values
        for j in range(1,n):
            S[:,j] = S[:,j-1] + (r-0.5*σ**2)*Δ + σ*np.sqrt(Δ)*Z[:,j-1]
            
        self.S = np.exp(S)  # Exponentiate to get the GBM paths
        return self.S
    
    ##########################
    ### Vectorized Version ###
    ##########################
    def GeometricBrownianMotion_vec(self):
        """ Generate GBM paths according to Algorithm 3.

        Returns:
        np.ndarray: Simulated paths of the asset price.
        """

        # unpack parameters
        Δ = self.Δ
        Z = self.Z
        S = self.S
        S0 = self.S0
        r = self.r
        σ = self.σ
        n = self.n

        # Generate all increments at once
        BM = (r - 0.5*σ**2)*Δ + σ*np.sqrt(Δ)*Z
        
        # Use cumsum to calculate the cumulative sum of increments and then exponentiate
        S[:,:] = np.log(S0)
        S[:,1:] += np.cumsum(BM, axis=1)

        # Multiply every path by the initial stock price
        self.S = np.exp(S)
        return self.S
    
    def MertonJumpDiffusion(self,α: float, β: float, λ: float):
        """
        Generate Merton Jump Diffusion paths according to Algorithm 4 assuming log-normal distribution of shocks.
        Parameters:
        α (float): Mean of log-normal jump size
        β (float): Volatility of log-normal jump size
        λ (float): Intensity rate of the Poisson process
        
        Returns:
        np.ndarray: Simulated paths of the asset price
        """
        self.α = α
        self.β = β
        self.λ = λ
        
        # unpack parameters
        Δ = self.Δ
        Z = self.Z
        S = self.S
        S0 = self.S0
        r = self.r
        σ = self.σ
        n = self.n
        α = self.α
        β = self.β
        λ = self.λ
        dim = self.dim

        S[:,0] = np.log(S0) 
        c = r - 0.5*σ**2 - λ*(np.exp(α + 0.5*β**2) - 1)
        
        # Generate Poisson and (log-)normal random jumps for all paths and time steps at once
        N = np.random.poisson(λ*Δ, (dim, n-1))  # Poisson process for the number of jumps
        Z_2 = np.random.normal(0, 1, (dim, n-1))  # Normal random variables for the jump sizes
        
        for j in range(1,n):
            # Compute jump sizes for each path
            M = α*N[:,j-1] + β*np.sqrt(N[:,j-1])*Z_2[:,j-1]
            # if no jump set jump process to zero 
            M = np.where(N[:,j-1] > 0, M, 0)
            # Calculate the combined diffusion and jump process
            S[:,j] = S[:,j-1] + c*Δ + σ*np.sqrt(Δ)*Z[:,j-1] + M
            
        self.S = np.exp(S) 
    
        return self.S
    
    ##########################
    ### Vectorized Version ###
    ##########################
    def MertonJumpDiffusion_vec(self, α: float, β: float, λ: float):
        """
        Generate Merton Jump Diffusion paths according to Algorithm 4 assuming log-normal distribution of shocks.
        Parameters:
        α (float): Mean of log-normal jump size
        β (float): Volatility of log-normal jump size
        λ (float): Intensity rate of the Poisson process
        
        Returns:
        np.ndarray: Simulated paths of the asset price
        """
        self.α = α
        self.β = β
        self.λ = λ
        
        # unpack parameters
        Δ = self.Δ
        Z = self.Z
        S = self.S
        S0 = self.S0
        r = self.r
        σ = self.σ
        n = self.n
        α = self.α
        β = self.β
        λ = self.λ
        dim = self.dim
        
        # No changes up to the definition of c
        c = r - 0.5*σ**2 - λ*(np.exp(α + 0.5*β**2) - 1)

        # Generate Poisson and (log-)normal random jumps for all paths and time steps at once
        N = np.random.poisson(λ*Δ, (dim, n-1))  # Poisson process for the number of jumps
        Z_2 = np.random.normal(0, 1, (dim, n-1))  # Normal random variables for the jump sizes

        # Calculate the jump sizes for all paths and time steps
        M = α * N + β*np.sqrt(N)*Z_2
        
        # if no jump set M = 0
        M = np.where(N > 0, M, 0)
        
        # Calculate the combined diffusion and jump process for all time steps
        S[:,:] = np.log(S0)
        S[:,1:] = np.log(S0) + np.cumsum(c*Δ + σ*np.sqrt(Δ)*Z + M, axis=1)

        self.S = np.exp(S)

        return self.S

    def CEV(self,γ: float):
        """
        Generate CEV paths according to Algorithm 5. 
        
        Parameters:
        γ (float): parameter governing elasticity with respect to price
        
        Returns:
        np.ndarray: Simulated paths of the asset price
        """ 
        assert γ>= 0, 'cant let elasticity be negative due to leverage effect'
        self.γ = γ

        # unpack parameters
        Δ = self.Δ
        Z = self.Z
        S = self.S
        S0 = self.S0
        r = self.r
        γ = self.γ
        σ = self.σ
        n = self.n

        S[:,0] = S0  # Set initial values
        # Simulation using the Euler-Maruyama method for the CEV model
        for j in range(1,n):
            S[:,j] = S[:,j-1] + r*S[:,j-1]*Δ + σ*S[:,j-1]**(γ/2)*np.sqrt(Δ)*Z[:,j-1]
        self.S = S

        return self.S
    
    
    def BS_option_value(self, otype: str = 'put'):
        ''' Closed-form valuation of a European option in Black-Scholes.
        
        Parameters:
        otype (str): Option type either call or put (defualt: put)
        
        Returns:
        float: Option price of a European put option
        '''
        
        # unpack 
        S0 = self.S0
        K = self.K
        r = self.r
        σ = self.σ
        T = self.T

        d1 = (np.log(S0/K) + (r + 0.5*σ**2)*T) / (σ*np.sqrt(T))
        d2 = d1 - σ*np.sqrt(T)
        
        if otype == 'call':
            value = (S0 * stats.norm.cdf(d1, 0., 1.) -
                 K * np.exp(-r * T)*stats.norm.cdf(d2, 0., 1.))
        elif otype == 'put':
            value = K * np.exp(-r*T)*stats.norm.cdf(-d2) - S0*stats.norm.cdf(-d1)
        else: 
            raise ValueError('Invalid option type.')
    
        return value
    def american_option_LSM(self, poly_degree: int, otype: str = 'put'):
        """
        American option pricing using the LSM as outlined in Algorithm 1.
        
        Parameters:
        poly_degree (int): x^n, number of basis functions
        otype (str): call or put (default)
        
        Returns:
        float: V0, LSM Estimator
        np.ndarray: Exercise times for each path (in case of early exercise)
        """
        
        assert otype == 'call' or otype == 'put', 'Invalid option type.'
        assert len(self.S) != 0, 'Please simulate a stock price process.'
        
        # unpack
        S = self.S
        K = self.K
        S0 = self.S0
        n = self.n
        dim = self.dim
        df = self.df
        
        # Initialize exercise_times array to store exercise times for each path
        exercise_times = np.full(dim, self.T)  # Initialize with T (no exercise)
        
        # Initialize an array to store payoffs
        payoffs = np.zeros(dim)

        # inner values
        if otype == 'call':
            self.intrinsic_val = np.maximum(S - K, 0)
        elif otype == 'put':
            self.intrinsic_val = np.maximum(K - S, 0)
            
        # last day cashflow == last day intrinsic value
        V = np.copy(self.intrinsic_val[:,-1])

        # Backward Induction
        for i in range(n - 2, 0, -1): # start at second to last and end at second to first
            # a. find itm path 
            # (potentially) better estimate the continuation value
            itm_path = np.where(self.intrinsic_val[:,i] > 0)  # evaluate: S[:,i] vs. K
            V = V * df # discount next period value
            V_itm = V[itm_path[0]] # define subset (note, we need to set [0] due to np.where being tuple)
            S_itm = S[itm_path[0],i]
            
            # b. run regression and calculate conditional expectation (LSM)
            # initialize continuation value
            C = np.zeros(shape=dim)
            # if only 5 itm paths (probably, otm options), then continuation value is zero
            if len(itm_path[0]) > 5:
                rg = np.polyfit(S_itm, V_itm, poly_degree)  # polynomial regression
                C[itm_path[0]] = np.polyval(rg, S_itm)  # evaluate conditional expectation
            
            # c. Calculation of value function at i 
            # if hold: V = 0, if exercise: V = intrinsic value
            exercise_condition = self.intrinsic_val[:, i] > C
            V = np.where(exercise_condition, self.intrinsic_val[:, i], V)
            
            for idx in np.where(exercise_condition)[0]:
                payoffs[idx] = self.intrinsic_val[idx, i]
                exercise_times[idx] = i * self.Δ
            
            # Update exercise times for paths that exercised
            exercise_times[exercise_condition] = i * self.Δ
            
        self.V0 = df * np.average(V)
        
        return self.V0, exercise_times
    def plot_paths(self):
        """
        Plot simulated stock price paths along with the mean path.
        """
        # Define time interval
        time = np.linspace(0, self.T, self.n)  # Ensure to include the initial time step

        # Calculate the mean of the paths at each time step
        mean_path = np.mean(self.S, axis=0)

        # Plot
        plt.figure(figsize=(10, 6))
        
        plt.plot(time, self.S.T, lw=1, alpha=0.25)

        # Plot the mean path with a higher alpha and a different color for visibility
        plt.plot(time, mean_path, 'b', lw=2, alpha=0.75, label='Mean Path')

        plt.xlabel("Time, $t$")
        plt.ylabel("Stock Price, $S_t$")
        plt.title(f'{self.dim} Stock Price Simulation Paths')
        plt.legend()
        
        plt.show()
        
    
        


In [54]:
# Initialize the Monte Carlo option pricing with a specific seed
mc_option_pricing = MonteCarloOptionPricing(r=0.06, S0=36., K=40., T=1., σ=0.2, dim=100000, n=50, seed=15001, use_AV=True)
# Generate 'dim' no. of stock price process paths
#simulated_paths = mc_option_pricing.GeometricBrownianMotion()
simulated_paths = mc_option_pricing.GeometricBrownianMotion_vec()

#simulated_paths = mc_option_pricing.MertonJumpDiffusion(α=-0.5,β=0.4,λ=0.4)
#simulated_paths = mc_option_pricing.MertonJumpDiffusion_vec(α=-0.5,β=0.4,λ=0.4)
#simulated_paths = mc_option_pricing.CEV(γ=1.5)

# plot stock price process
#mc_option_pricing.plot_paths() 

Matching Longstaff table 1 "Simulated American" (100,000 simulations (50,000 plus 50,000 antithetic), exercisable 50 times a year)

In [55]:
mc_option_pricing.american_option_LSM(poly_degree = 3, otype= 'put')

(4.471997785451471, array([0.04, 1.  , 1.  , ..., 0.14, 1.  , 0.04]))

Below: Mathcing Longstaff table 1 row 1 for "Closed Form European"

In [56]:
mc_option_pricing.BS_option_value(otype='put')

3.84430779159684

## 2.1 Producing *American put prices* similar to *Simulated American* in table 1 in Longstaff.
- Where the simulation is based on 100000 (50000 plus 50000 antithetic) paths for the stock price.

In [57]:
# Parameters
r = 0.06  # risk-free rate
K = 40.   # strike price
dim = 100000  # number of simulation paths
n = 50  # number of exercise rights per year
seed = 15001  # random seed for reproducibility
use_AV = True  # antithetic variates
poly_degree = 3  # polynomial degree for LSM regression

# Option parameters
S_values = [36, 38, 40, 42, 44]
sigma_values = [0.20, 0.40]
T_values = [1, 2]

# Store results
results = []

# Iterate over each combination of S, sigma, and T
for S in S_values:
    for sigma in sigma_values:
        for T in T_values:
            # Initialize the Monte Carlo pricer
            mc_option_pricing = MonteCarloOptionPricing(r, S, K, T, sigma, dim, n, seed, use_AV)
            # Simulate the asset paths
            mc_option_pricing.GeometricBrownianMotion_vec()
            # Price the option using LSM
            option_price = mc_option_pricing.american_option_LSM(poly_degree, otype='put')
            # Store the results
            results.append((S, sigma, T, option_price))
            # Print individual values without formatting the entire tuple
            print(f"S: {S}, Sigma: {sigma}, T: {T}, American Put Option Price: {option_price[0]:.4f}")



S: 36, Sigma: 0.2, T: 1, American Put Option Price: 4.4720
S: 36, Sigma: 0.2, T: 2, American Put Option Price: 4.8259
S: 36, Sigma: 0.4, T: 1, American Put Option Price: 7.0751
S: 36, Sigma: 0.4, T: 2, American Put Option Price: 8.4595
S: 38, Sigma: 0.2, T: 1, American Put Option Price: 3.2400
S: 38, Sigma: 0.2, T: 2, American Put Option Price: 3.7267
S: 38, Sigma: 0.4, T: 1, American Put Option Price: 6.1222
S: 38, Sigma: 0.4, T: 2, American Put Option Price: 7.6295
S: 40, Sigma: 0.2, T: 1, American Put Option Price: 2.3049
S: 40, Sigma: 0.2, T: 2, American Put Option Price: 2.8727
S: 40, Sigma: 0.4, T: 1, American Put Option Price: 5.2752
S: 40, Sigma: 0.4, T: 2, American Put Option Price: 6.8784
S: 42, Sigma: 0.2, T: 1, American Put Option Price: 1.6036
S: 42, Sigma: 0.2, T: 2, American Put Option Price: 2.1992
S: 42, Sigma: 0.4, T: 1, American Put Option Price: 4.5470
S: 42, Sigma: 0.4, T: 2, American Put Option Price: 6.1980
S: 44, Sigma: 0.2, T: 1, American Put Option Price: 1.09

## 2.1 Producing *American put prices* with our own parameter values


In [62]:
# Parameters
r = 0.04  # risk-free rate
K = 100.   # strike price
dim = 100000  # number of simulation paths
n = 50  # number of exercise rights per year
seed = 15001  # random seed for reproducibility
use_AV = True  # antithetic variates
poly_degree = 3  # polynomial degree for LSM regression

# Option parameters
S_values = [90, 95, 100, 105, 110]
sigma_values = [0.20, 0.40]
T_values = [1, 2]

# Store results
results = []

# Iterate over each combination of S, sigma, and T
for S in S_values:
    for sigma in sigma_values:
        for T in T_values:
            # Initialize the Monte Carlo pricer
            mc_option_pricing = MonteCarloOptionPricing(r, S, K, T, sigma, dim, n, seed, use_AV)
            # Simulate the asset paths
            mc_option_pricing.GeometricBrownianMotion_vec()
            # Price the option using LSM
            option_price = mc_option_pricing.american_option_LSM(poly_degree, otype='put')
            # Store the results
            results.append((S, sigma, T, option_price))
            # Print individual values without formatting the entire tuple
            print(f"S: {S}, Sigma: {sigma}, T: {T}, American Put Option Price: {option_price[0]:.4f}")



S: 90, Sigma: 0.2, T: 1, American Put Option Price: 11.7552
S: 90, Sigma: 0.2, T: 2, American Put Option Price: 13.0895
S: 90, Sigma: 0.4, T: 1, American Put Option Price: 18.4854
S: 90, Sigma: 0.4, T: 2, American Put Option Price: 22.6126
S: 95, Sigma: 0.2, T: 1, American Put Option Price: 8.7427
S: 95, Sigma: 0.2, T: 2, American Put Option Price: 10.3973
S: 95, Sigma: 0.4, T: 1, American Put Option Price: 16.0652
S: 95, Sigma: 0.4, T: 2, American Put Option Price: 20.5113
S: 100, Sigma: 0.2, T: 1, American Put Option Price: 6.3504
S: 100, Sigma: 0.2, T: 2, American Put Option Price: 8.2211
S: 100, Sigma: 0.4, T: 1, American Put Option Price: 13.9399
S: 100, Sigma: 0.4, T: 2, American Put Option Price: 18.5763
S: 105, Sigma: 0.2, T: 1, American Put Option Price: 4.5206
S: 105, Sigma: 0.2, T: 2, American Put Option Price: 6.4477
S: 105, Sigma: 0.4, T: 1, American Put Option Price: 12.0817
S: 105, Sigma: 0.4, T: 2, American Put Option Price: 16.8341
S: 110, Sigma: 0.2, T: 1, American Pu

# 3.1 Finite Difference results (benchmark for analysis) - Longstaff

In [58]:
import scipy.sparse
import scipy.sparse.linalg
import pandas as pd

def Amerput_implicit(S, K, r, sigma, T, N, M):
    dt = T / N
    ds = 2 * S / M
    A = scipy.sparse.lil_matrix((M+1, M+1))
    f = np.maximum(K - np.arange(0, M+1) * ds, 0)

    for m in range(1, M):
        x = 1 / (1 - r * dt)
        A[m, m-1] = x * (r * m * dt - sigma**2 * m**2 * dt) / 2
        A[m, m] = x * (1 + sigma**2 * m**2 * dt)
        A[m, m+1] = x * (-r * m * dt - sigma**2 * m**2 * dt) / 2

    A[0, 0] = 1
    A[M, M] = 1

    for i in range(N, 0, -1):
        f = scipy.sparse.linalg.spsolve(A.tocsr(), f)
        f = np.maximum(f, K - np.arange(0, M+1) * ds)

    P = f[round((M+1) / 2)]
    return P
# Function to calculate prices for different combinations
def calculate_prices(combinations):
    results = []
    for combo in combinations:
        S, sigma, T = combo
        price = Amerput_implicit(S, K, r, sigma, T, N*T, M)
        results.append({
            "S": S,
            "Sigma": sigma,
            "Maturity": T,
            "Price": price
        })
    return results
# Parameters
K = 40  # strike price
r = 0.06  # short-term interest rate
N = 50  # number of time steps per year
M = 1000  # spatial steps

# Combinations of S, sigma, and T
combinations = [
    (36, 0.20, 1), (36, 0.20, 2), (36, 0.40, 1), (36, 0.40, 2),
    (38, 0.20, 1), (38, 0.20, 2), (38, 0.40, 1), (38, 0.40, 2),
    (40, 0.20, 1), (40, 0.20, 2), (40, 0.40, 1), (40, 0.40, 2),
    (42, 0.20, 1), (42, 0.20, 2), (42, 0.40, 1), (42, 0.40, 2),
    (44, 0.20, 1), (44, 0.20, 2), (44, 0.40, 1), (44, 0.40, 2)
]

# Calculate prices
results = calculate_prices(combinations)

# Convert results to a DataFrame for better visualization
df = pd.DataFrame(results)
print(df)

     S  Sigma  Maturity     Price
0   36    0.2         1  4.464959
1   36    0.2         2  4.829426
2   36    0.4         1  7.075359
3   36    0.4         2  8.414515
4   38    0.2         1  3.236798
5   38    0.2         2  3.734485
6   38    0.4         1  6.122398
7   38    0.4         2  7.593174
8   40    0.2         1  2.301416
9   40    0.2         2  2.875075
10  40    0.4         1  5.287724
11  40    0.4         2  6.855543
12  42    0.2         1  1.606117
13  42    0.2         2  2.203934
14  42    0.4         1  4.559553
15  42    0.4         2  6.193117
16  44    0.2         1  1.101470
17  44    0.2         2  1.682617
18  44    0.4         1  3.926463
19  44    0.4         2  5.598173


# 3.2 Finite Difference results (benchmark for analysis) - our parameters

In [65]:
import scipy.sparse
import scipy.sparse.linalg
import pandas as pd

def Amerput_implicit(S, K, r, sigma, T, N, M):
    dt = T / N
    ds = 2 * S / M
    A = scipy.sparse.lil_matrix((M+1, M+1))
    f = np.maximum(K - np.arange(0, M+1) * ds, 0)

    for m in range(1, M):
        x = 1 / (1 - r * dt)
        A[m, m-1] = x * (r * m * dt - sigma**2 * m**2 * dt) / 2
        A[m, m] = x * (1 + sigma**2 * m**2 * dt)
        A[m, m+1] = x * (-r * m * dt - sigma**2 * m**2 * dt) / 2

    A[0, 0] = 1
    A[M, M] = 1

    for i in range(N, 0, -1):
        f = scipy.sparse.linalg.spsolve(A.tocsr(), f)
        f = np.maximum(f, K - np.arange(0, M+1) * ds)

    P = f[round((M+1) / 2)]
    return P
# Function to calculate prices for different combinations
def calculate_prices(combinations):
    results = []
    for combo in combinations:
        S, sigma, T = combo
        price = Amerput_implicit(S, K, r, sigma, T, N*T, M)
        results.append({
            "S": S,
            "Sigma": sigma,
            "Maturity": T,
            "Price": price
        })
    return results
# Parameters
K = 100  # strike price
r = 0.04  # short-term interest rate
N = 50  # number of time steps per year
M = 1000  # spatial steps

# Combinations of S, sigma, and T
combinations_our = [
    (90, 0.20, 1), (90, 0.20, 2), (90, 0.40, 1), (90, 0.40, 2),
    (95, 0.20, 1), (95, 0.20, 2), (95, 0.40, 1), (95, 0.40, 2),
    (100, 0.20, 1), (100, 0.20, 2), (100, 0.40, 1), (100, 0.40, 2),
    (105, 0.20, 1), (105, 0.20, 2), (105, 0.40, 1), (105, 0.40, 2),
    (110, 0.20, 1), (110, 0.20, 2), (110, 0.40, 1), (110, 0.40, 2)
]

# Calculate prices
results_our = calculate_prices(combinations_our)

# Convert results to a DataFrame for better visualization
df = pd.DataFrame(results_our)
print(df)

      S  Sigma  Maturity      Price
0    90    0.2         1  11.763679
1    90    0.2         2  13.105444
2    90    0.4         1  18.513853
3    90    0.4         2  22.494832
4    95    0.2         1   8.743665
5    95    0.2         2  10.426930
6    95    0.4         1  16.109825
7    95    0.4         2  20.426013
8   100    0.2         1   6.363961
9   100    0.2         2   8.241844
10  100    0.4         1  13.987530
11  100    0.4         2  18.550345
12  105    0.2         1   4.540519
13  105    0.2         2   6.475259
14  105    0.4         1  12.122525
15  105    0.4         2  16.851292
16  110    0.2         1   3.180050
17  110    0.2         2   5.059168
18  110    0.4         1  10.490050
19  110    0.4         2  15.313117


# 4.1 European put Black-Scholes - Longstaff parameters

In [68]:
# Parameters
r = 0.06  # risk-free rate
K = 40.   # strike price
dim = 100000  # number of simulation paths
n = 50  # number of exercise rights per year
seed = 15001  # random seed for reproducibility
use_AV = True  # antithetic variates
poly_degree = 3  # polynomial degree for LSM regression

# Option parameters
S_values = [36, 38, 40, 42, 44]
sigma_values = [0.20, 0.40]
T_values = [1, 2]

# Storing results
option_prices = {}

for S0 in S_values:
    for σ in sigma_values:
        for T in T_values:
            pricing_model = MonteCarloOptionPricing(r, S0, K, T, σ, dim, n, seed, use_AV)
            option_price = pricing_model.BS_option_value('put')
            option_prices[(S0, σ, T)] = option_price

option_prices


{(36, 0.2, 1): 3.84430779159684,
 (36, 0.2, 2): 3.7630009276697347,
 (36, 0.4, 1): 6.711399066623681,
 (36, 0.4, 2): 7.700039587695633,
 (38, 0.2, 1): 2.8519321180394748,
 (38, 0.2, 2): 2.9905568579925728,
 (38, 0.4, 1): 5.834320866481415,
 (38, 0.4, 2): 6.978802476201395,
 (40, 0.2, 1): 2.066401004420346,
 (40, 0.2, 2): 2.3558662816645324,
 (40, 0.4, 1): 5.059623125933808,
 (40, 0.4, 2): 6.325998988916153,
 (42, 0.2, 1): 1.4645039410864467,
 (42, 0.2, 2): 1.8413544774586459,
 (42, 0.4, 1): 4.3787183635270335,
 (42, 0.4, 2): 5.735617602382007,
 (44, 0.2, 1): 1.0169152264282708,
 (44, 0.2, 2): 1.4292151307875187,
 (44, 0.4, 1): 3.78279883264722,
 (44, 0.4, 2): 5.201995311264419}

# 4.2 European put Black-Scholes - Own parameters


In [69]:
# Parameters
r = 0.04  # risk-free rate
K = 100.   # strike price
dim = 100000  # number of simulation paths
n = 50  # number of exercise rights per year
seed = 15001  # random seed for reproducibility
use_AV = True  # antithetic variates
poly_degree = 3  # polynomial degree for LSM regression

# Option parameters
S_values = [90, 95, 100, 105, 110]
sigma_values = [0.20, 0.40]
T_values = [1, 2]

# Storing results
option_prices = {}

for S0 in S_values:
    for σ in sigma_values:
        for T in T_values:
            pricing_model = MonteCarloOptionPricing(r, S0, K, T, σ, dim, n, seed, use_AV)
            option_price = pricing_model.BS_option_value('put')
            option_prices[(S0, σ, T)] = option_price

option_prices


{(90, 0.2, 1): 10.841383007448584,
 (90, 0.2, 2): 11.44837100524984,
 (90, 0.4, 1): 17.98179956710704,
 (90, 0.4, 2): 21.47631169879711,
 (95, 0.2, 1): 8.161821798465631,
 (95, 0.2, 2): 9.242460996086365,
 (95, 0.4, 1): 15.690289107394975,
 (95, 0.4, 2): 19.539350254125694,
 (100, 0.2, 1): 6.003997632506753,
 (100, 0.2, 2): 7.396337485348852,
 (100, 0.4, 1): 13.65723072051609,
 (100, 0.4, 2): 17.77797309922748,
 (105, 0.2, 1): 4.321292350264937,
 (105, 0.2, 2): 5.872228544355089,
 (105, 0.4, 1): 11.862467598482496,
 (105, 0.4, 2): 16.17778689967097,
 (110, 0.2, 1): 3.0476219457362106,
 (110, 0.2, 2): 4.62918195500102,
 (110, 0.4, 1): 10.284942477887839,
 (110, 0.4, 2): 14.725065425174904}